# Exploring 10 Year Non Shopp Data
* [Using Smart Sheet.](https://app.smartsheet.com/workspaces/8MgpHcXR4GJVM5GvWWMmQg7M8gqhJj88Gfh54Pr1)

In [75]:
import numpy as np
import pandas as pd
from calitp import *

In [76]:
pd.options.display.max_columns = 100
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

In [77]:
import _utils

In [78]:
GCS_FILE_PATH = "gs://calitp-analytics-data/data-analyses/project_prioritization/"

## Investment Plan

In [79]:
# Open first sheet.
df_investment_plan = to_snakecase(
    pd.read_excel(f"{GCS_FILE_PATH}10 Year Non-SHOPP Investment Plan.xlsx")
)

/opt/conda/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default


### Look at null values for each column

In [80]:
df_investment_plan.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 717 entries, 0 to 716
Data columns (total 58 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   ppno                                     217 non-null    object 
 1   ct_project_id                            312 non-null    object 
 2   district                                 713 non-null    float64
 3   county                                   715 non-null    object 
 4   route                                    713 non-null    object 
 5   beg_pm                                   330 non-null    object 
 6   end_pm                                   329 non-null    object 
 7   urban_rural                              631 non-null    object 
 8   project_name                             717 non-null    object 
 9   ea                                       333 non-null    object 
 10  current_phase                            396 non-n

### Some Cleaning

In [81]:
df_investment_plan["total_project_cost__$1,000_"] = df_investment_plan[
    "total_project_cost__$1,000_"
].replace(
    {
        "Phase 1 (SR-60 to Mission Blvd) - $174,000\nPhase 2 (Mission Blvd to I-10) - $170,000": 344000,
        "$85,738 (Cycle 2)": 85738,
        "Oxnard to Camarillo 2nd Main Track: $15,000\nOxnard Station North Platform: $20,000": 35000,
        "$214,000\nPhase 1: $82,000\nPhase 2: $132,000": 214000,
        "$-": 0,
        "TBD": 0,
    }
)

In [82]:
# Fill in total project cost with fake number
df_investment_plan["total_project_cost__$1,000_"] = (
    df_investment_plan["total_project_cost__$1,000_"]
    .astype("float64")
    .fillna(888888888.0)
)

In [83]:
# Coerce dates
date_cols = [
    "pid_approval_date",
    "rtl_date",
    "con_start_date",
]
for c in date_cols:
    df_investment_plan[c] = df_investment_plan[c].apply(pd.to_datetime, errors="coerce")

In [84]:
# Fill na based on types
df_investment_plan = df_investment_plan.fillna(
    df_investment_plan.dtypes.replace({"float64": 0.0, "object": "None"})
)

In [85]:
# Lowercase string columns
string_cols = [
    "county",
    "route",
    "potential_funding_program_s_",
    "lead_agency",
    "primary_mode",
    "urban_rural",
    "current_phase",
    "project_name",
    "potential_funding_program_s_",
]

In [86]:
for c in string_cols:
     df_investment_plan[c] = df_investment_plan[c].str.title().str.strip()

In [87]:
# Correct some curent phase values that read in weirdly
df_investment_plan["current_phase"] = df_investment_plan["current_phase"].replace(
    {
        "ps&e\nr/w": "ps&e",
        "other\nplan/concept": "other\plan\concept",
        "pid\nplan/concept": "pid\plan\concept",
        "ps&e\nrow": "ps&e\row",
    }
)

### Duplicated PPNO?

In [88]:
# Check that each row represents a different project
# PPNO and Project ID have a lot less unique value b/c 70% and 56% of each col are not filled in
for i in ["ppno", "ct_project_id", "project_name"]:
    print(f"{i} : {df_investment_plan[i].nunique()}")

ppno : 204
ct_project_id : 300
project_name : 716


In [89]:
# One project is included twice
df_investment_plan["project_name"].value_counts().head()

Delano To Pixely 6-Lane                                                                        2
I-5 Btw Rte 73 And Sd Co, Managed Lanes Project                                                1
San Diego - Coronado Bay Bridge Suicide Deterrent Project                                      1
Linking Neighborhoods Of La Jolla Reservation With Bike/Walking Trail On The State Route 76    1
Sr-78 Hov/Managed Lanes                                                                        1
Name: project_name, dtype: int64

In [90]:
# Check and make sure there aren't duplicate PPNO-Project Name combos
duplicate_ppno = (
    df_investment_plan[~df_investment_plan["ppno"].isin([0, "None"])]
    .groupby("ppno")
    .agg({"project_name": "count"})
    .sort_values("project_name")
    .reset_index()
)

In [91]:
# Filter out for ppno with more than 2 project names.
# Save those projects into a list
duplicate_ppno = duplicate_ppno[duplicate_ppno["project_name"] > 1]

In [92]:
duplicate_ppno_list = duplicate_ppno["ppno"].to_list()

In [93]:
# Check out these rows in our df
# are these duplicates?
len(df_investment_plan[df_investment_plan["ppno"].isin(duplicate_ppno_list)])

24

In [94]:
duplicate_projects_df = df_investment_plan[
    df_investment_plan["ppno"].isin(duplicate_ppno_list)
]

In [95]:
duplicate_projects_df[
    ["ppno", "project_name", "current_phase", "ea", "project_description"]
]

,ppno,project_name,current_phase,ea,project_description
159,2830,Us 101 Avila Beach Dr. Interchange,Ps&E,1G480,"Construct operational improvements and a park-and-ride lot. A roundabout is proposed to be constructed at the intersection of the US 101 southbound ramps, Avila Beach Dr., and Shell Beach Rd. Operational and-or additional safety enhancements will be considered for the US 101 northbound ramps-Monte Rd. intersection."
162,2830,Us 101/ Avila Beach Rd I/C Improvement And Park-And-Ride Lot,Ps&E\nR/W,1G480,Interchange operational improvements for US 101 SB ramps at Avila Beach Dr. at Shell Beach Rd. Construct Park-and-Ride Lot at southwest corner of Avila Beach Dr. and Shell Beach Rd near Pismo Beach
178,6288,South Fresno I/C Project,Pa&Ed,0H240,Modify interchanges
182,7036,Sr 180 West Extension,Candidate,0Y230,"In Fresno County, near Mendota from I-5 to Route 33, construct 2-lane conventional highway on new alignment. This project would propose to construct new alignments and covert existing County roadways to a State Route."
184,7036,180 West Extension,Pid,0Y230,Construct highway
189,6288,South Fresno State Route 99 Corridor Project,Pa&Ed,0H240,"In Fresno County, in and near Fresno, from 0.4 mile south of American Avenue to 0.4 mile north of North Avenue. Modify interchanges."
193,6694,Grapevine I/C,Pa&Ed,0R100,Relocate I/C
194,3386E,Sr 46 Gap Closure Segment 4C,Ps&E/Row,44256,"From Brown Material Rd to Farnsworth Ave, 2C to 4E"
203,6694,Grapevine Interchange,None,0R100,"In Kern County, near Grapevine from 0.7 mile south of the Grapevine UC to 0.5 mile south of the California Aqueduct, relocate interchange.\nThis project is mitigation to the Grapevine Specific and Community Plan EIR."
206,3386E,Sr-46 Expressway - Segment 4C (Kern)\nSr 46 Gap Closure Segment 4C,Con,44256,"44256 In and near Lost Hills, from 1 mile west of Browns Material Road to California Aquaduct. Convert from 2-lane conventional highway to 4-lane expressway.\n\nFrom Brown Material Road to Farnsworth Avenue, convert 2-lane conventional highway to a 4-lane expressway. Possibly drop if it is fully funded."


### Add Fake Values

In [96]:
df_investment_plan2 = df_investment_plan[
    [
        "ppno",
        "ct_project_id",
        "district",
        "route",
        "urban_rural",
        "project_name",
        "current_phase",
        "project_description",
        "priority_project",
        "previous_caltrans_nominations",
        "primary_mode",
        "lead_agency",
        "pid_approval_date",
        "target_pa_ed",
        "rtl_date",
        "con_start_date",
        "funding_need_phase_s_",
        "total_project_cost__$1,000_",
        "potential_funding_program_s_"
    ]
]

In [97]:
# https://stackoverflow.com/questions/64093880/how-to-create-random-floats-and-add-them-as-a-dataframe-column
np.random.seed(365)

In [98]:
# Fake columns
fake_columns = [
    "increase_peak_person_throughput",
    "reduction_in_peak_period_delay",
    "reduction_in_fatal_and_injury_crashes",
    "reduction_in_injury_rates",
    "increase_access_to_jobs",
    "increase_access_jobs_to_DAC",
    "commercial_dev_developed",
    "tons_of_goods_impacted",
    "improve_air_quality",
    "impact_natural_resources",
    "support_of_trasnportation",
]

In [99]:
for i in fake_columns:
    df_investment_plan2[i] = np.round(
        np.random.uniform(0.0, 100.0, size=(len(df_investment_plan), 1)), 2
    )

/tmp/ipykernel_1161/4138376327.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/tmp/ipykernel_1161/4138376327.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/tmp/ipykernel_1161/4138376327.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/tmp/ipykernel_1161/41383

In [100]:
df_investment_plan2['fake_fund_requested'] = np.round(
        np.random.uniform(0.0, 100000.0, size=(len(df_investment_plan), 1)), 2
    )

/tmp/ipykernel_1161/2532840627.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [101]:
df_investment_plan2['fake_benefit_score'] = ((df_investment_plan2[fake_columns].sum(axis=1))/(df_investment_plan2['fake_fund_requested']))*10000

/tmp/ipykernel_1161/1026160082.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [102]:
df_investment_plan2['statewide_rank'] = df_investment_plan2['fake_benefit_score'].rank(ascending=False)

/tmp/ipykernel_1161/2278500973.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [103]:
df_investment_plan2['district_rank'] = df_investment_plan2.groupby('district')['fake_benefit_score'].rank(ascending=False)

/tmp/ipykernel_1161/3740384342.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [104]:
df_investment_plan2.sample()

,ppno,ct_project_id,district,route,urban_rural,project_name,current_phase,project_description,priority_project,previous_caltrans_nominations,primary_mode,lead_agency,pid_approval_date,target_pa_ed,rtl_date,con_start_date,funding_need_phase_s_,"total_project_cost__$1,000_",potential_funding_program_s_,increase_peak_person_throughput,reduction_in_peak_period_delay,reduction_in_fatal_and_injury_crashes,reduction_in_injury_rates,increase_access_to_jobs,increase_access_jobs_to_DAC,commercial_dev_developed,tons_of_goods_impacted,improve_air_quality,impact_natural_resources,support_of_trasnportation,fake_fund_requested,fake_benefit_score,statewide_rank,district_rank
317,None,None,7.0,Off,Urban,La County Freight Rail Efficiency Project At Malabar Yard,None,The project would replace intermodal railcar storage capacity from BNSF's West Bank Yard to BNSF's Malabar Yard. Approximately 500 feet of new track would be constructed to connect BNSF Malabar Yard with the Los Angeles Railway Junction through east 46th Street. The 49th Street at-grade railroad crossing would be permanently closed.,0.0,DRMT Consolidated Statewide Project List\nINFRA 2021 Caltrans Application\nStimulus Drill 2021\nTCEP Cycle 2 Priority 6 of 26 (Not Awarded),Rail (Freight),Bnsf/La Metro,datetime64[ns],None,2023-04-30 00:00:00,datetime64[ns],None,85738.0,Crisi\nInfra\nIntercity Pass Rail\nTcep,55.5,82.94,28.26,74.65,91.23,84.87,80.99,57.89,43.97,62.56,14.05,44124.84,153.407922,235.0,28.0


In [105]:
df_investment_plan2.to_excel(f"{GCS_FILE_PATH}fake_data.xlsx", sheet_name='fake', index= False)  

### Some Analysis
#### Are projects in rural or urban areas? 
* What defines rural versus urban?

In [106]:
rural_urban_projects = _utils.value_counts_df(df_investment_plan, "urban_rural").rename(
    columns={"urban_rural": "Total Projects", "index": "Urban/Rural"}
)

In [107]:
rural_urban_projects

,Urban/Rural,Total Projects
0,Urban,528
1,Rural,103
2,None,86


In [108]:
_utils.basic_bar_chart(
    rural_urban_projects, "Urban/Rural", "Total Projects", "Urban/Rural"
)

alt.Chart(...)

#### Most Common Lead Agencies.

In [109]:
# Filter out for lead agency that says "none"
df_investment_plan[df_investment_plan["lead_agency"] != "none"].groupby(
    ["lead_agency"]
).agg({"project_name": "count"}).sort_values("project_name", ascending=False).head(
    10
).style.bar(
    subset=["project_name"], color="#8CBCCB"
).set_properties(
    **{"background-color": "white"}
).set_table_styles(
    [dict(selector="th", props=[("text-align", "center")])]
).set_properties(
    **{"text-align": "center"}
)

,project_name
lead_agency,
None,204
Caltrans,163
Sjjpa/Sjrrc,54
Lossan,17
Metrolink,13
Octa/Caltrans,12
Sandag,11
La Metro,9
Smart,9


#### Primary Mode

In [110]:
_utils.value_counts_df(df_investment_plan, "primary_mode").rename(
    columns={"primary_mode": "Total Projects", "index": "Primary Mode"}
).sort_values("Total Projects", ascending=False).head(10).style.bar(
    subset=["Total Projects"], color="#8CBCCB"
).set_properties(
    **{"background-color": "white"}
).set_table_styles(
    [dict(selector="th", props=[("text-align", "center")])]
).set_properties(
    **{"text-align": "center"}
)

,Primary Mode,Total Projects
0,Highway,203
1,Rail (Passenger),122
2,Bike/Pedestrian,65
3,Interchange (Modification),62
4,Rail (Freight),37
5,Interchange (New),33
6,Interchange (Widening),32
7,Grade Crossing,32
8,Complete Streets,27
9,Port,26


#### Costs

In [111]:
(
    df_investment_plan.groupby("primary_mode")
    .agg({"total_project_cost__$1,000_": "sum"})
    .sort_values("total_project_cost__$1,000_", ascending=False)
)

,"total_project_cost__$1,000_"
primary_mode,
Highway,7.113790e+10
Interchange (Modification),2.222413e+10
Bike/Pedestrian,1.955810e+10
Interchange (New),1.866799e+10
Interchange (Widening),1.777868e+10
Transit,7.113068e+09
Rail (Passenger),6.272636e+09
Rail (Freight),6.228358e+09
Grade Crossing,6.224379e+09
